In [1]:
import pandas as pd
import os
from sklearn.preprocessing import LabelEncoder
import warnings
import numpy as np
from custom_libraries.load_data_for_label import load_data_for_label
from custom_libraries.merge_data import merge_data
from pycaret.classification import *
from custom_libraries.find_peaks import find_peaks_cust

In [ ]:
pip install pycaret

# Load Test Data

In [2]:
folder_path = "LABELED"
os.makedirs(folder_path, exist_ok=True)
file_names = []
for name in os.listdir(folder_path):
    file_path = f"{folder_path}/{name}"
    file_names.append(file_path)
#Ahora tengo que especificar mis features 
signals = ["accX", "accY", "accZ", "gyroX", "gyroY", "gyroZ", "magnX", "magnY", "magnZ", "linAccX", "linAccY", "linAccZ"]
data = load_data_for_label(filelist=file_names[0:2], signals= signals)
df= pd.concat(data)

In [3]:
df.shape

(14696, 37)

In [4]:
#X = df.drop(['peaks', 'first_sample','last_sample', 'first_sample_closest_peak', 
 #            'last_sample_closest_peak','exercise','exercising_periods'], axis=1)
X = df.copy()
X.columns

Index(['accX_orig', 'accY_orig', 'accZ_orig', 'gyroX_orig', 'gyroY_orig',
       'gyroZ_orig', 'magnX_orig', 'magnY_orig', 'magnZ_orig', 'linAccX_orig',
       'linAccY_orig', 'linAccZ_orig', 'accX', 'accY', 'accZ', 'gyroX',
       'gyroY', 'gyroZ', 'magnX', 'magnY', 'magnZ', 'linAccX', 'linAccY',
       'linAccZ', 'accX_mod', 'accY_mod', 'accZ_mod', 'gyroX_mod', 'gyroY_mod',
       'gyroZ_mod', 'magnX_mod', 'magnY_mod', 'magnZ_mod', 'linAccX_mod',
       'linAccY_mod', 'linAccZ_mod', 'peaks'],
      dtype='object')

# Auxiliary Functions

In [5]:
def prepare_data(chunk_df):
    signals = ["accX", "accY", "accZ", "gyroX", "gyroY", "gyroZ", "magnX", "magnY", "magnZ", "linAccX", "linAccY", "linAccZ"]
    stats_dict = {}
    series = chunk_df[signals]
    for column in series.columns:
        mean = series[column].mean()
        std = series[column].std()
        median = series[column].median()
    
        stats_dict[f"{column}_mean"] = mean
        stats_dict[f"{column}_std"] = std
        stats_dict[f"{column}_median"] = median
        
    return pd.DataFrame.from_dict(stats_dict, orient="index").T

# Load Model

In [6]:
# Load the model
pipeline = load_model(model_name="squats_traditional")

Transformation Pipeline and Model Successfully Loaded


In [7]:
ventana = prepare_data(X.iloc[0:51])
prediction = predict_model(pipeline, ventana, raw_score=True)
prediction.prediction_label.iloc[0]

'NO_EXERCISE'

# Check Data

In [8]:
nuevo_df_predicciones = pd.DataFrame()
nuevo_df_list = []  # Lista para almacenar las filas

for idx, fila in df.iterrows():
    warnings.filterwarnings('ignore')
    nuevo_df_list.append(fila.copy())  # Añadir la fila a la lista
    
    if pd.notna(fila['peaks']):
        nuevo_df = pd.DataFrame(nuevo_df_list)  # Convertir la lista en DataFrame
        X1 = prepare_data(nuevo_df)
        prediction = predict_model(pipeline, X1, raw_score=True)
        y_pred = prediction.prediction_label.iloc[0]
        X1['start'] = nuevo_df.index[0]
        X1['end'] = nuevo_df.index[-1]
        X1['prediccion'] = y_pred
        nuevo_df_predicciones = pd.concat([nuevo_df_predicciones, X1], ignore_index=True)
        
        # Limpiar la lista para la próxima ventana
        nuevo_df_list = [fila.copy()]

In [9]:
nuevo_df_predicciones.shape

(102, 39)

In [10]:
nuevo_df_predicciones.head()

,accX_mean,accX_std,accX_median,accY_mean,accY_std,accY_median,accZ_mean,accZ_std,accZ_median,gyroX_mean,...,linAccX_median,linAccY_mean,linAccY_std,linAccY_median,linAccZ_mean,linAccZ_std,linAccZ_median,start,end,prediccion
0,0.229717,0.067639,0.244986,8.058149,1.554099,8.962906,1.570357,0.878528,1.993743,35.269588,...,-0.183683,0.240955,0.086366,0.229795,-1.557238,1.642113,-0.622280,1970-01-01 00:00:00.000,1970-01-01 00:00:00.474,NO_EXERCISE
1,-0.807387,0.383328,-0.863861,9.315241,2.270159,10.122217,-5.358753,2.847514,-5.475630,6.446957,...,0.604449,-0.064382,0.201904,-0.017634,1.084862,3.311060,1.841822,1970-01-01 00:00:00.474,1970-01-01 00:00:01.567,SQUAT
2,0.076474,0.160205,0.093024,8.715241,1.331371,9.495311,1.061530,0.926204,1.576240,-0.930495,...,-0.006470,0.113929,0.109336,0.102512,-0.973412,1.357032,-0.161967,1970-01-01 00:00:01.567,1970-01-01 00:00:03.012,NO_EXERCISE
3,-0.843387,0.375272,-0.789857,9.008479,1.701445,9.499292,-5.602087,2.786894,-5.682057,0.362799,...,0.641912,0.050774,0.227051,0.083403,0.956848,2.846323,1.384450,1970-01-01 00:00:03.012,1970-01-01 00:00:04.203,SQUAT
4,0.097256,0.142006,0.139285,8.755725,1.254845,9.467373,1.058756,0.881370,1.455207,-0.470624,...,-0.012081,0.091856,0.121684,0.077902,-0.942312,1.289586,-0.226361,1970-01-01 00:00:04.203,1970-01-01 00:00:05.666,NO_EXERCISE


In [ ]:
np.unique(nuevo_df_predicciones.prediccion,return_counts=True)

In [11]:
squats_detection = nuevo_df_predicciones[nuevo_df_predicciones.prediccion == 'SQUAT'][['start', 'end', 'prediccion']]
squats_detection

,start,end,prediccion
1,1970-01-01 00:00:00.474,1970-01-01 00:00:01.567,SQUAT
3,1970-01-01 00:00:03.012,1970-01-01 00:00:04.203,SQUAT
5,1970-01-01 00:00:05.666,1970-01-01 00:00:06.799,SQUAT
7,1970-01-01 00:00:08.378,1970-01-01 00:00:09.650,SQUAT
9,1970-01-01 00:00:11.172,1970-01-01 00:00:12.532,SQUAT
11,1970-01-01 00:00:14.170,1970-01-01 00:00:15.347,SQUAT
13,1970-01-01 00:00:16.757,1970-01-01 00:00:17.858,SQUAT
15,1970-01-01 00:00:19.277,1970-01-01 00:00:20.503,SQUAT
17,1970-01-01 00:00:22.083,1970-01-01 00:00:23.233,SQUAT
19,1970-01-01 00:00:24.916,1970-01-01 00:00:25.955,SQUAT


In [12]:
squats_detection = nuevo_df_predicciones[nuevo_df_predicciones.prediccion == ('NO_EXERCISE'
                                                                              ''
                                                                              '')][['start', 'end', 'prediccion']]
squats_detection

,start,end,prediccion
0,1970-01-01 00:00:00.000,1970-01-01 00:00:00.474,NO_EXERCISE
2,1970-01-01 00:00:01.567,1970-01-01 00:00:03.012,NO_EXERCISE
4,1970-01-01 00:00:04.203,1970-01-01 00:00:05.666,NO_EXERCISE
6,1970-01-01 00:00:06.799,1970-01-01 00:00:08.378,NO_EXERCISE
8,1970-01-01 00:00:09.650,1970-01-01 00:00:11.172,NO_EXERCISE
10,1970-01-01 00:00:12.532,1970-01-01 00:00:14.170,NO_EXERCISE
12,1970-01-01 00:00:15.347,1970-01-01 00:00:16.757,NO_EXERCISE
14,1970-01-01 00:00:17.858,1970-01-01 00:00:19.277,NO_EXERCISE
16,1970-01-01 00:00:20.503,1970-01-01 00:00:22.083,NO_EXERCISE
18,1970-01-01 00:00:23.233,1970-01-01 00:00:24.916,NO_EXERCISE
